<a href="https://colab.research.google.com/github/cbedart/S-DISCO/blob/main/S_DISCO_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<center><h1>S-DISCO - Machine Learning</h1></center>**

---



## <u>**Installation of mandatory packages**</u>

In [ ]:
# @title Installation of prerequisites
!pip install rdkit

import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdFingerprintGenerator, Draw
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

from sklearn.decomposition import PCA, KernelPCA
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import datasets
from sklearn.cluster import KMeans

from sklearn import tree
from sklearn.tree import export_graphviz, DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut

from sklearn.preprocessing import StandardScaler

import ipywidgets as widgets
from IPython.display import Image, display, clear_output, Javascript

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import plotly.express as px
import pydotplus

################################################################################

launch_rf_hyperparameters = 0

################################################################################

def tree_graph_to_png(tree, feature_names, class_names, png_file_to_save):
    tree_str = export_graphviz(tree, feature_names=feature_names, class_names=class_names,
                                     filled=True, out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)
    graph.write_png(png_file_to_save)


def getMolDescriptors(mol, missingVal=None):
    res = {}
    for nm,fn in Descriptors._descList:
        try:
            val = fn(mol)
        except:
            val = missingVal
        res[nm] = val
    return res

################################################################################

print("\n\n\033[1mPrerequisites succesfully installed !\033[0m")

In [ ]:
# @title Loading data from the previous session

!wget https://github.com/cbedart/S-DISCO/raw/main/qsar_dict.pkl > /dev/null 2>&1
qsar_dict = pd.read_pickle("qsar_dict.pkl")

################################################################################

desc_GR_DTs = qsar_dict["desc_clustered_prep"]

pic50_limit = 8

ygroup2 = []
ycat2 = []
for i in desc_GR_DTs.iloc[:,2]:
  if i > pic50_limit:
    ygroup2.append("Active")
    ycat2.append(1)
  else:
    ygroup2.append("Less Active")
    ycat2.append(0)

desc_GR_activity2 = pd.concat([desc_GR_DTs.iloc[:,:4], pd.DataFrame(ygroup2, columns=["Activity_Label"]), pd.DataFrame(ycat2, columns=["Activity"]), desc_GR_DTs.iloc[:,4:]], axis = "columns")

X2 = desc_GR_activity2.iloc[:,6:]
y2 = desc_GR_activity2.loc[:,"Activity"]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

################################################################################

print("Consistency in splitting your data for training and validation is crucial\nfor reliable model evaluation and comparison. Using the same split ensures\nthat your evaluation metrics are consistent across different model iterations,\nmaking it easier to compare their performance.")
print("\nHere, we will do a single data split, with :")
print("- A train set containing 80% of the data ({} compounds)".format(len(X_train2)))
print("- A test set containing 20% of the data ({} compounds)".format(len(X_test2)))

print("\nWe will also setup the pIC50 limit to {}, with :".format(pic50_limit))
print("- A total of {} active compounds (pIC50 < {}) - Coded as 1".format(desc_GR_activity2["Activity"].value_counts()[1], pic50_limit))
print("- A total of {} less active compounds (pIC50 > {}) - Coded as 0".format(desc_GR_activity2["Activity"].value_counts()[0], pic50_limit))

print("\n\033[1mDone !\033[0m \n\n")


display(desc_GR_DTs)


## <u>**Model \#3 - Decision tree**</u>

In [ ]:
# @title First decision tree

dt1 = DecisionTreeClassifier()
dt1.fit(X_train2, y_train2)

y_pred_dt1 = dt1.predict(X_test2)
# dt1_confusion = pd.DataFrame(confusion_matrix(y_test2, y_pred_dt1), columns=["Less Actives", "Actives"], index=["Less Actives", "Actives"])

print("Accuracy on the train set = {}\n".format(round(dt1.score(X_train2, y_train2),3)))
print("Accuracy on the test set = {}".format(round(dt1.score(X_test2, y_test2),3)))

cvk_dt1 = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scores_cvk_dt1 = cross_val_score(dt1, X2, y2, scoring='accuracy', cv=cvk_dt1, n_jobs=-1)

print("Accuracy using 10-fold Cross Validation = {}\n".format(round(np.mean(scores_cvk_dt1),3)))


tree_graph_to_png(dt1, feature_names=X2.columns, class_names=["Less Actives", "Actives"], png_file_to_save='dt1.png')

print("Confusion matrix:")
fig, axn = plt.subplots(1,2, figsize=(12,3))
axn[0].grid(False) ; axn[1].grid(False)
confusion1_dt1 = ConfusionMatrixDisplay.from_estimator(dt1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[0], cmap=plt.cm.Blues, normalize=None)
confusion1_dt1.ax_.set_title("Confusion matrix, without normalization")
# confusion1_dt1
confusion2_dt1 = ConfusionMatrixDisplay.from_estimator(dt1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[1], cmap=plt.cm.Blues, normalize="true")
confusion2_dt1.ax_.set_title("Normalized confusion matrix")
# confusion2_dt1
plt.show()

dt1_acc_pr_rec = [round(dt1.score(X_train2, y_train2),3),
                  round(dt1.score(X_test2, y_test2),3),
                  round(confusion2_dt1.confusion_matrix[1,1] / (confusion2_dt1.confusion_matrix[1,1] + confusion2_dt1.confusion_matrix[1,0]), 3),
                  round(confusion2_dt1.confusion_matrix[1,1] / (confusion2_dt1.confusion_matrix[1,1] + confusion2_dt1.confusion_matrix[0,1]), 3)]

In [ ]:
# @title Display the first decision tree

# @markdown Do not hesitate to right clic >>> "Open in a new tab" to better see the tree

display(Image('dt1.png'))

## <u>**Model \#4 - Random forest**</u>

In [ ]:
# @title Build the first random forest model

# @markdown Choose the number of decision trees you will generate:
number_of_trees = 100 # @param {type:"integer"}

# @markdown Choose if you set an early termination of the trees after X splits, or just build the bigger trees you can (= None)
maximum_depth = "None" # @param ["None", 1, 2, 3, 4, 5]

if maximum_depth == "None":
  maximum_depth = None
else:
  maximum_depth = int(maximum_depth)

print("\n\033[1mUsing {} decision trees with an early termination setup at {}\033[0m\n".format(number_of_trees, maximum_depth))

rf1 = RandomForestClassifier(n_estimators = number_of_trees, max_depth = maximum_depth, random_state=42)
rf1.fit(X_train2, y_train2)

y_pred_rf1 = rf1.predict(X_test2)
rf1_confusion = pd.DataFrame(confusion_matrix(y_test2, y_pred_rf1), columns=["Less Actives", "Actives"], index=["Less Actives", "Actives"])

print("Accuracy on the train set = {}\n".format(round(rf1.score(X_train2, y_train2),3)))
print("Accuracy on the test set = {}".format(round(rf1.score(X_test2, y_test2),3)))

cvk_rf1 = RepeatedStratifiedKFold(n_splits=5, n_repeats = 3, random_state=1)
scores_cvk_rf1 = cross_val_score(rf1, X2, y2, scoring='accuracy', cv=cvk_rf1, n_jobs=-1)

print("Accuracy using 5-fold Cross Validation = {}\n".format(round(np.mean(scores_cvk_rf1),3)))

print("Confusion matrix:")
fig, axn = plt.subplots(1,2, figsize=(12,3))
axn[0].grid(False) ; axn[1].grid(False)
confusion1_rf1 = ConfusionMatrixDisplay.from_estimator(rf1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[0], cmap=plt.cm.Blues, normalize=None)
confusion1_rf1.ax_.set_title("Confusion matrix, without normalization")
# confusion1_rf1
confusion2_rf1 = ConfusionMatrixDisplay.from_estimator(rf1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[1], cmap=plt.cm.Blues, normalize="true")
confusion2_rf1.ax_.set_title("Normalized confusion matrix")
# confusion2_rf1
plt.show()

rf1_acc_pr_rec = [round(rf1.score(X_train2, y_train2),3),
                  round(rf1.score(X_test2, y_test2),3),
                  round(confusion2_rf1.confusion_matrix[1,1] / (confusion2_rf1.confusion_matrix[1,1] + confusion2_rf1.confusion_matrix[1,0]), 3),
                  round(confusion2_rf1.confusion_matrix[1,1] / (confusion2_rf1.confusion_matrix[1,1] + confusion2_rf1.confusion_matrix[0,1]), 3)]

In [ ]:
# @title Hyperparameters optimization of the random forest model

if launch_rf_hyperparameters == 1:
  raise FileExistsError('Already done')

rf_accuracies_train = []
rf_accuracies_test = []
rf_precision_test = []
rf_recall_test = []


for maximum_depth_loop in [1, 2, 3, 4, 5, None]:
  for number_of_trees_loop in [10,100,1000]:
  # for number_of_trees_loop in [10,100,1000,10000]:
    print("Currently running depth {} with {} trees".format(maximum_depth_loop, number_of_trees_loop))
    rfloop = RandomForestClassifier(n_estimators = number_of_trees_loop, max_depth = maximum_depth_loop, random_state=42)
    rfloop.fit(X_train2, y_train2)
    y_pred_rfloop = rfloop.predict(X_test2)
    rfloop_confusion = confusion_matrix(y_test2, y_pred_rfloop, normalize="true")

    rf_accuracies_train.append(round(rfloop.score(X_train2, y_train2),3))
    rf_accuracies_test.append(round(rfloop.score(X_test2, y_test2),3))
    rf_precision_test.append(round(rfloop_confusion[1,1] / (rfloop_confusion[1,1] + rfloop_confusion[1,0]), 3))
    rf_recall_test.append(round(rfloop_confusion[1,1] / (rfloop_confusion[1,1] + rfloop_confusion[0,1]), 3))


print("\n\033[1mDone !\033[0m")

In [ ]:
# @title Plot for the hyperparameters optimization

plt.figure(figsize=(10,5))
plt.plot(rf_accuracies_train, marker=".")
plt.plot(rf_accuracies_test, marker=".")
plt.plot(rf_precision_test, marker=".")
plt.plot(rf_recall_test, marker=".")
plt.title("Quality assessment metrics for RF hyperparameters optimization")
plt.legend(["Accuracy train", "Accuracy test", "Precision", "Recall"])
plt.ylim([-0.05,1.05])

labels = []
for maximum_depth_loop in [1, 2, 3, 4, 5, None]:
  # for number_of_trees_loop in [10,100,1000,10000]:
  for number_of_trees_loop in [10,100,1000]:
    labels.append("{} / {}".format(maximum_depth_loop, number_of_trees_loop))

plt.xticks(range(18), labels, rotation='vertical')
plt.show()

## <u>**Model \#5 - Artificial neural networks**</u>

In [ ]:
# @title Build your own artificial neural network
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 700})'''))

# @markdown Number of hidden layers:
nb_layers = 1 # @param [1,2,3,4,5] {type:"raw"}

# @markdown Number of nodes per hidden layers:
layer1 = 100 # @param {type:"integer"}
layer2 = 100 # @param {type:"integer"}
layer3 = 100 # @param {type:"integer"}
layer4 = 100 # @param {type:"integer"}
layer5 = 100 # @param {type:"integer"}

# @markdown Activation function:
activation_function = "logistic" # @param ["identity", "logistic", "tanh", "relu"] {type:"string"}

# @markdown Solver for weight optimization :
solver_opt = "adam" # @param ["adam", "lbfgs"] {type:"string"}

################################################################################

if nb_layers == 1:
  if layer1 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1)
elif nb_layers == 2:
  if layer1 <= 0 or layer2 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1, layer2)
elif nb_layers == 3:
  if layer1 <= 0 or layer2 <= 0 or layer3 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1, layer2, layer3)
elif nb_layers == 4:
  if layer1 <= 0 or layer2 <= 0 or layer3 <= 0 or layer4 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1, layer2, layer3, layer4)
else :
  if layer1 <= 0 or layer2 <= 0 or layer3 <= 0 or layer4 <= 0 or layer5 <= 0:
    raise ValueError('Wrong setup - You have a wrong number nodes for some of hidden layers')
  custom_layer_sizes=(layer1, layer2, layer3, layer4, layer5)

nn1 = MLPClassifier(hidden_layer_sizes=custom_layer_sizes, activation = activation_function, verbose = True, solver = solver_opt, alpha=1e-4, max_iter= 2000)
# nn1 = MLPClassifier(hidden_layer_sizes=custom_layer_sizes, activation = 'identity', verbose = True, solver = 'lbfgs', alpha=1e-5, max_iter= 2000)
nn1 = nn1.fit(X_train2, y_train2)
y_pred_nn1 = nn1.predict(X_test2)

################################################################################

print("\n\n\033[1mQuality assessment metrics - {} hidden layers / {} architecture:\033[0m\n".format(nb_layers, custom_layer_sizes))
print("Accuracy on the train set = {}\n".format(round(nn1.score(X_train2, y_train2),3)))
print("Accuracy on the test set = {}\n".format(round(nn1.score(X_test2, y_test2),3)))

print("Confusion matrix:")
fig, axn = plt.subplots(1,2, figsize=(12,3))
axn[0].grid(False) ; axn[1].grid(False)
confusion1_nn1 = ConfusionMatrixDisplay.from_estimator(nn1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[0], cmap=plt.cm.Blues, normalize=None)
confusion1_nn1.ax_.set_title("Confusion matrix, without normalization")
# confusion1_nn1
confusion2_nn1 = ConfusionMatrixDisplay.from_estimator(nn1, X_test2, y_test2, display_labels=["Less Active", "Active"], ax=axn[1], cmap=plt.cm.Blues, normalize="true")
confusion2_nn1.ax_.set_title("Normalized confusion matrix")
# confusion2_nn1
plt.show()

print("\nPrecision = {}".format(round(confusion2_nn1.confusion_matrix[1,1] / (confusion2_nn1.confusion_matrix[1,1] + confusion2_nn1.confusion_matrix[1,0]), 3)))
print("\nRecall = {}".format(round(confusion2_nn1.confusion_matrix[1,1] / (confusion2_nn1.confusion_matrix[1,1] + confusion2_nn1.confusion_matrix[0,1]), 3)))

nn1_acc_pr_rec = [round(nn1.score(X_train2, y_train2),3),
                  round(nn1.score(X_test2, y_test2),3),
                  round(confusion2_nn1.confusion_matrix[1,1] / (confusion2_nn1.confusion_matrix[1,1] + confusion2_nn1.confusion_matrix[1,0]), 3),
                  round(confusion2_nn1.confusion_matrix[1,1] / (confusion2_nn1.confusion_matrix[1,1] + confusion2_nn1.confusion_matrix[0,1]), 3)]

## <u>**New compounds prediction !**</u>

In [ ]:
# @title 4 new compounds


# CHEMBL4081121 - Ki:  0.0800nM  / Ki:  3.30nM
# "Cc1cccc(S(=O)(=O)N2CCC3=Cc4c(cnn4-c4ccc(F)cc4)C[C@]3(C(=O)c3cc(C)ccn3)C2)c1"

# CHEMBL247048 - Ki:  3.70E+3nM
# "O=C1C=C2CCN(Cc3ccccc3)CC2(Cc2ccccc2)CC1"

# CHEMBL4088286 - Ki: 0.120nM / Ki: 9.20nM
# "O=C(c1ccccn1)[C@]12Cc3cnn(-c4ccc(F)cc4)c3C=C1CCN(S(=O)(=O)c1cc(F)cc(F)c1)C2"

# CHEMBL3734774 - Ki:  0.160nM / Ki:  100nM
# "O=C(c1ccccn1)[C@]12Cc3cnn(-c4ccc(F)cc4)c3C=C1CCN(S(=O)(=O)c1ccc(C(F)(F)F)cc1)C2"

desc_new = pd.DataFrame(columns=["Compound ID"] + [i for i,j in Descriptors._descList[:-1]])
cpds_new = ["Compound_1", "Compound_2","Compound_3","Compound_4"]
smiles_new = ["Cc1cccc(S(=O)(=O)N2CCC3=Cc4c(cnn4-c4ccc(F)cc4)C[C@]3(C(=O)c3cc(C)ccn3)C2)c1",
              "O=C1C=C2CCN(Cc3ccccc3)CC2(Cc2ccccc2)CC1",
              "O=C(c1ccccn1)[C@]12Cc3cnn(-c4ccc(F)cc4)c3C=C1CCN(S(=O)(=O)c1cc(F)cc(F)c1)C2",
              "O=C(c1ccccn1)[C@]12Cc3cnn(-c4ccc(F)cc4)c3C=C1CCN(S(=O)(=O)c1ccc(C(F)(F)F)cc1)C2"]

for i in range(len(smiles_new)):
  mol_new = Chem.AddHs(Chem.MolFromSmiles(smiles_new[i]))
  desc_new.loc[i] = [cpds_new[i]] + list(getMolDescriptors(mol_new, missingVal = np.NAN).values())

display(Draw.MolsToGridImage([Chem.MolFromSmiles(smiles_new[i]) for i in range(len(smiles_new))],molsPerRow=4,subImgSize=(200,200),legends=cpds_new))

display(desc_new)

In [ ]:
# @title Scaled using the same methods applied previously for the train/test dataset
desc_train_normal = qsar_dict['desc_clustered']
desc_train_normal_filtered = desc_train_normal.loc[desc_train_normal["BindingDB_ID"].isin(desc_GR_DTs["BindingDB_ID"])]
desc_train_normal_filtered = desc_train_normal_filtered[desc_GR_DTs.columns]
desc_train_normal_filtered

scaler = qsar_dict["scaler"]
desc_new = desc_new[list(scaler.feature_names_in_)]
# scaler.fit(desc_train_normal_filtered.iloc[:,4:])

desc_new_scaled = pd.DataFrame(scaler.transform(desc_new))
desc_new_scaled.columns = list(scaler.feature_names_in_)
desc_new_scaled = desc_new_scaled[X_train2.columns.to_list()]

desc_new_scaled = pd.concat([pd.DataFrame(cpds_new), desc_new_scaled], axis="columns")
desc_new_scaled.columns = ["Compound ID"] + X_train2.columns.to_list()
desc_new_scaled

In [ ]:
# @title Predictions time

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda2 = LinearDiscriminantAnalysis()
lda2 = lda2.fit(X_train2, y_train2)
y_pred_lda2 = lda2.predict(X_test2)
confusion2_lda2 = confusion_matrix(y_test2, y_pred_lda2, normalize="true")
lda2_acc_pr_rec = [round(lda2.score(X_train2, y_train2),3),
                  round(lda2.score(X_test2, y_test2),3),
                  round(confusion2_lda2[1,1] / (confusion2_lda2[1,1] + confusion2_lda2[1,0]), 3),
                  round(confusion2_lda2[1,1] / (confusion2_lda2[1,1] + confusion2_lda2[0,1]), 3)]

a1 = lda2.predict(desc_new_scaled.iloc[:,1:])
a2 = dt1.predict(desc_new_scaled.iloc[:,1:])
a3 = rf1.predict(desc_new_scaled.iloc[:,1:])
a4 = nn1.predict(desc_new_scaled.iloc[:,1:])

new_results = pd.DataFrame([a1, a2, a3, a4], index=["LDA", "DT", "RF", "NN"], columns=cpds_new).replace(0, "Less active").replace(1, "Active")
rf1_acc_pr_rec
new_results[["Accuracy train", "Accuracy test", "Precision", "Recall"]] = lda2_acc_pr_rec
new_results.loc["LDA",["Accuracy train", "Accuracy test", "Precision", "Recall"]] = dt1_acc_pr_rec
new_results.loc["DT",["Accuracy train", "Accuracy test", "Precision", "Recall"]] = dt1_acc_pr_rec
new_results.loc["RF",["Accuracy train", "Accuracy test", "Precision", "Recall"]] = rf1_acc_pr_rec
new_results.loc["NN",["Accuracy train", "Accuracy test", "Precision", "Recall"]] = nn1_acc_pr_rec
new_results